In [2]:
from datasets import load_dataset, Dataset, DatasetDict, Audio

In [37]:
RECORDINGS_DIR="/run/media/johnny/31c5407a-2da6-4ef8-95ec-d294c1afec38/" 
META_FILE="../malorca/metadata_dev12.json"
SAVE_DIR="malorca"

# load the metadata
# meta_data = json.load(open(META_FILE, mode='r'))
dataset = load_dataset("json", data_files=META_FILE, split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [38]:
# set properly path to the recordings according to the current disk path
dataset = dataset.map(lambda x: {"audio": RECORDINGS_DIR + x["audio"]}, remove_columns=["audio"])

Map:   0%|          | 0/6335 [00:00<?, ? examples/s]

In [39]:
dataset[0]

{'audio': '/run/media/johnny/31c5407a-2da6-4ef8-95ec-d294c1afec38/MALORCA/DATA_ATC/VIENNA/WAV_FILES/LOWW07/BALAD_20160701_B7_1/2016-07-01__05-30-12-01.wav',
 'full_ts': 'austrian four five five november continue on present heading climb flight level two four zero',
 'short_ts': 'austrian 455N continue on present heading climb flight level 240 \n'}

In [40]:
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [41]:
dataset.save_to_disk(SAVE_DIR)

Saving the dataset (0/1 shards):   0%|          | 0/6335 [00:00<?, ? examples/s]

In [31]:
dd = Dataset.load_from_disk(SAVE_DIR)

In [32]:
dd

Dataset({
    features: ['audio', 'full_ts', 'short_ts'],
    num_rows: 3987
})

In [33]:
dd[0]

{'audio': {'path': '2016-07-01__05-30-12-01.wav',
  'array': array([-5.45940537e-04, -2.77002080e-04,  5.77722967e-04, ...,
         -1.16415322e-10,  2.91038305e-11,  2.91038305e-11]),
  'sampling_rate': 16000},
 'full_ts': 'austrian four five five november continue on present heading climb flight level two four zero',
 'short_ts': 'austrian 455N continue on present heading climb flight level 240 \n'}

In [9]:
import whisper
model = whisper.load_model("medium")

In [ ]:

# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio("/run/media/johnny/31c5407a-2da6-4ef8-95ec-d294c1afec38/ATCO2-ASRdataset-v1_final/DATA_nonEN/LKPR_RUZYNE_Tower_134_560MHz_20201025_110406.wav")
audio = whisper.pad_or_trim(audio)

# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)

# detect the spoken language
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

Detected language: cs


In [15]:
from glob import glob
from pydub import AudioSegment
from bs4 import BeautifulSoup, ResultSet
from tqdm import tqdm  
import os 

def is_english_lang(xml_data):
    # returns True if more than 50% of the segments are in English, otherwise False
    soup = BeautifulSoup(xml_data, "xml")
    english_segments = 0
    num_of_segments = len(soup.find_all("segment"))
    for segment in soup.find_all("segment"):
        tags = segment.find('tags')
        if (tags):
            tag_en = tags.find("non_english")
            if (tag_en): # the tag is <non_english></non_english>, meaning 0 is english
                english_segments += 1 - int(tag_en.get_text())
    
    if (num_of_segments == 0):
        return True    
    else: 
        return english_segments / num_of_segments > 0.5

count = 0
langs = {}

folders = ["/run/media/johnny/31c5407a-2da6-4ef8-95ec-d294c1afec38/ATCO2-ASRdataset-v1_final/DATA_nonEN/",
           "/run/media/johnny/31c5407a-2da6-4ef8-95ec-d294c1afec38/ATCO2-ASRdataset-v1_final/DATA-data-nonEN/"]

for folder in folders:
    for file in tqdm(glob(folder + '*.wav')):
        # load audio and pad/trim it to fit 30 seconds
        audio = whisper.load_audio(file)
        audio = whisper.pad_or_trim(audio)

        # make log-Mel spectrogram and move to the same device as the model
        mel = whisper.log_mel_spectrogram(audio).to(model.device)

        # detect the spoken language
        _, probs = model.detect_language(mel)
        if (max(probs, key=probs.get) == "en"):
            probs.pop("en")
        # print(f"Detected language: {max(probs, key=probs.get)}", file)
        langs[max(probs, key=probs.get)] = langs.get(max(probs, key=probs.get), 0) + 1

print("SET OF LANGUAGES: ")
for lang in langs:
    print(lang, langs[lang]) 

# for file in tqdm(glob('/run/media/johnny/31c5407a-2da6-4ef8-95ec-d294c1afec38/ATCO2-ASRdataset-v1_final/DATA_nonEN/*.xml')):
#     if (is_english_lang(open(file, mode='r').read())):
#         count += 1
#         file = file.removesuffix('.xml')
#         !mv {file}.* /run/media/johnny/31c5407a-2da6-4ef8-95ec-d294c1afec38/ATCO2-ASRdataset-v1_final/DATA_nonEN-datanonen-EN/

# print(count) 

    # wav = AudioSegment.from_wav(file)
    # # wav length in seconds
    # length = len(wav) / 1000
    # if length > 30:
    #     print(file, length)
        # file = file.removesuffix('.wav')
        # print("MOVING: ", file)
        
        

100%|██████████| 31/31 [00:11<00:00,  2.63it/s]

SET OF LANGUAGES: 
de 8
fr 113
cs 20
nl 2
pl 4
nn 1
sk 6


In [9]:
sss = """
Map: 100%
 105/105 [00:02<00:00, 62.80 examples/s]
Map: 100%
 1579/1579 [00:46<00:00, 67.74 examples/s]
Map: 100%
 92/92 [00:01<00:00, 62.47 examples/s]
Map: 100%
 4032/4032 [01:48<00:00,  6.20 examples/s]
Map: 100%
 6335/6335 [02:57<00:00, 68.19 examples/s]
Map: 100%
 1373/1373 [00:45<00:00, 62.45 examples/s]
Map: 100%
 10706/10706 [05:15<00:00, 71.56 examples/s]
"""
sss = sss.split("\n")
sss = [int(x.strip().split('/')[0]) for x in sss if x.find("Map") == -1 and x != ""]
train_sum = sum(sss)
print(train_sum)

vvv = """
Map: 100%
 70/70 [00:01<00:00, 57.88 examples/s]
Map: 100%
 53/53 [00:01<00:00, 54.97 examples/s]
Map: 100%
 412/412 [00:10<00:00,  6.78 examples/s]
Map: 100%
 33/33 [00:00<00:00, 65.98 examples/s]
Map: 100%
 40/40 [00:01<00:00, 46.83 examples/s]
Map: 100%
 800/800 [00:19<00:00, 55.90 examples/s]
Map: 100%
 400/400 [00:08<00:00, 66.78 examples/s]
Map: 100%
 300/300 [00:05<00:00, 71.85 examples/s]
Map: 100%
 1557/1557 [00:47<00:00,  2.47 examples/s]
Map: 100%
 492/492 [00:15<00:00,  3.82 examples/s]
Map: 100%
 3961/3961 [01:47<00:00, 62.96 examples/s]
"""
vvv = vvv.split("\n")
vvv = [int(x.strip().split('/')[0]) for x in vvv if x.find("Map") == -1 and x != ""]
test_sum = sum(vvv)
print(test_sum)

24222
8118
